## Data Extracting Process

#### 解决click element not find的问题: 用ActionChain, 翻页后再点击
`
ElementClickInterceptedException: Message: element click intercepted: Element <span aria-hidden="true" class="icon--24-chevron-right-v2 navigation-button-icon__373c0__2Fl7a css-12anxc3">...</span> is not clickable at point (222, 588). Other element would receive the click: <html xmlns:fb="http://www.facebook.com/2008/fbml" class="js" lang="en">...</html>
  (Session info: headless chrome=88.0.4324.150)
 `

#### Text cleaning checkpoints
- Check out for null/empty/missing values
- Remove characters
    - punctuaion
    - `\n`, `\t`, `\r`等
    - special characters like `!@#$%^&*()`
    - 有些文本有那种`\a0`什么，估计是emoji吧
    - numbers are usually consider useless
    - Contraction thing 有缩写字典，应该还好。这个可以处理。
- Lower case
- Spaces: double space -> single space

上面的基本预处理完之后，再建立`nltk`/`gensim`/`sklearn`的文本处理模型。
- BoW
- TF-IDF

\* 注：这些文本矩阵要结合餐厅category来训练。

#### Other cleaning
- Date and datetime

In [1]:
import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys    # type things in the search bar and see the results

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC

import time
from bs4 import BeautifulSoup
import time
from lxml import etree

PATH = '/Users/liqingran/Downloads/chromedriver'

from selenium.webdriver.chrome.options import Options   # 实现无可视化界面
from selenium.webdriver import ChromeOptions            # 实现selenium规避检测

# 实例化options并进行参数设定，实现无可视化界面
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')

# 实现检测规避，这个不懂怎么加，再说吧
option = ChromeOptions()
option.add_experimental_option('excludeSwitches', ['enable-automation'])
# driver = Chrome(options=option)

In [11]:
def retrive_review(parser):
    """Retrive real-time reviews for each dynamic source page."""
    info = []
    reviews = []
    ratings = []
    dates = []
    name = parser.xpath('//*[@id="wrap"]/div[3]/yelp-react-root/div/div[2]/div[1]/div[1]/div/div/div[1]/h1/text()')
    li_list = parser.xpath('//*[@id="wrap"]/div[3]/yelp-react-root/div/div[3]/div/div/div[2]/div/div[1]/div[2]/section[2]/div[2]/div/ul/li')
    for li in li_list:
        review = li.xpath('./div/div[3]/p/span/text()')
        rating = li.xpath('./div/div[2]/div/div[1]/span/div/@aria-label')
        date = li.xpath('./div/div[2]/div/div[2]/span/text()')    
        if review:    # 因为如果这个review没放图片就在div[3]，如果放了图片就在div[4]
            reviews.append(review[0])
        else:
            review = li.xpath('./div/div[4]/p/span/text()')
            reviews.append(review[0])
        ratings.append(rating[0])
        dates.append(date[0])

    for i in range(len(reviews)):
        dict_ = {
            "Name": name[0],
            "Rating": ratings[i],
            "Date": dates[i],
            "Review": reviews[i]       
        }
        info.append(dict_)        
    return info


def get_max_page(parser):
    """Get the total page number for further click iteration."""
    num = parser.xpath('//*[@id="wrap"]/div[3]/yelp-react-root/div/div[3]/div/div/div[2]/div/div[1]/div[2]/section[2]/div[2]/div/div[4]/div[2]/span/text()')
    num = int(num[0].split(' ')[-1])
    return num


def organize_info(info):
    """Since each time we return lists of dictionaries, 
       this function is for concatinating all dictionaries into one single list."""
    final_info = []
    for i in info:
        for j in i:
            final_info.append(j)
    return final_info


def next_page_button(parser):
    """Find the element of the clicking next page button."""
    button_xpath = '//*[@id="wrap"]/div[3]/yelp-react-root/div/div[3]/div/div/div[2]/div/div[1]/div[2]/section[2]/div[2]/div/div[4]/div[1]/div/div'
    page_div = parser.xpath(button_xpath)
    click_point = len(page_div)
    click_xpath = button_xpath + str([click_point]) + '/span/a/span'
    return click_xpath


def scrap_review(url):
#     PATH = '/Users/liqingran/Downloads/chromedriver'
    driver = webdriver.Chrome(executable_path = PATH, options = chrome_options)

    res = driver.get(url)
    time.sleep(5)

    parser = etree.HTML(str(driver.page_source).replace('<br>', '\n').replace('</br>', '\n'))
    total_page_num = get_max_page(parser)
    
    next_page_xpath = next_page_button(parser)

    info = []
    # 点击 total_page_num-1 次，因为最后一页有可能不够20个review，我不想要了
    if total_page_num < 10:
        for i in range(total_page_num-1):
            dict_ = retrive_review(parser)
            info.append(dict_)
            print("->Page", i+1, "sucess!")
#             driver.find_element_by_xpath(next_page_xpath).click()      # 找到下一页的span标签并点击
            element = driver.find_element_by_xpath(next_page_xpath)
            webdriver.ActionChains(driver).move_to_element(element).click(element).perform()
            time.sleep(5)                                             
            parser = etree.HTML(str(driver.page_source).replace('<br>', '\n').replace('</br>', '\n'))
    else:
        for i in range(9):
            dict_ = retrive_review(parser)
            info.append(dict_)
            print("->Page", i+1, "sucess!")
            element = driver.find_element_by_xpath(next_page_xpath)
            webdriver.ActionChains(driver).move_to_element(element).click(element).perform()
            time.sleep(5)
            parser = etree.HTML(str(driver.page_source).replace('<br>', '\n').replace('</br>', '\n'))
#             try:
#                 driver.find_element_by_xpath(next_page_xpath).click() 
#                 time.sleep(10)  
#                 parser = etree.HTML(str(driver.page_source).replace('<br>', '\n').replace('</br>', '\n'))  
#             except:
#                 print("Click Interception, break on page", i+1)
#             finally:
#                 driver.quit()
#                 return info
  
    driver.quit()
    return info


In [6]:
# Test
url_test = 'https://www.yelp.com/biz/capt-loui-fort-lee-2'
info = scrap_review(url_test)

->Page 1 sucess!
->Page 2 sucess!
->Page 3 sucess!
->Page 4 sucess!
->Page 5 sucess!
->Page 6 sucess!
->Page 7 sucess!
->Page 8 sucess!
->Page 9 sucess!


In [12]:
# Test for more businesses al-together
url_list = ['https://www.yelp.com/biz/gopchang-story-fort-lee-fort-lee',
             'https://www.yelp.com/biz/soup-dumpling-plus-fort-lee',
             'https://www.yelp.com/biz/soba-noodle-azuma-fort-lee-2',
             'https://www.yelp.com/biz/sa-rit-gol-fort-lee-2',
             'https://www.yelp.com/biz/lauren-s-chicken-burger-fort-lee',
             'https://www.yelp.com/biz/gamja-tang-tang-fort-lee',
             'https://www.yelp.com/biz/oiso-bbq-pit-fort-lee-2',
             'https://www.yelp.com/biz/capt-loui-fort-lee-2',
             'https://www.yelp.com/biz/martys-fort-lee',
             'https://www.yelp.com/biz/sushi-kai-fort-lee',
             'https://www.yelp.com/biz/wok-bar-fort-lee']

In [13]:
total_info = []
for i in range(len(url_list)):
    info = scrap_review(url_list[i])
    info = organize_info(info)
    total_info.append(info)
    print("The", i+1, "business scraping success!")

->Page 1 sucess!
The 1 business scraping success!
->Page 1 sucess!
->Page 2 sucess!
->Page 3 sucess!
->Page 4 sucess!
->Page 5 sucess!
->Page 6 sucess!
->Page 7 sucess!
->Page 8 sucess!
->Page 9 sucess!
The 2 business scraping success!
->Page 1 sucess!
->Page 2 sucess!
->Page 3 sucess!
->Page 4 sucess!
->Page 5 sucess!
->Page 6 sucess!
->Page 7 sucess!
->Page 8 sucess!
->Page 9 sucess!
The 3 business scraping success!
->Page 1 sucess!
->Page 2 sucess!
->Page 3 sucess!
->Page 4 sucess!
->Page 5 sucess!
->Page 6 sucess!
->Page 7 sucess!
->Page 8 sucess!
The 4 business scraping success!
->Page 1 sucess!
The 5 business scraping success!
->Page 1 sucess!
->Page 2 sucess!
->Page 3 sucess!
->Page 4 sucess!
->Page 5 sucess!
->Page 6 sucess!
->Page 7 sucess!
->Page 8 sucess!
->Page 9 sucess!
The 6 business scraping success!
->Page 1 sucess!
The 7 business scraping success!
->Page 1 sucess!
->Page 2 sucess!
->Page 3 sucess!
->Page 4 sucess!
->Page 5 sucess!
->Page 6 sucess!
->Page 7 sucess!
->Pa

In [14]:
total_info = organize_info(total_info)

In [15]:
len(total_info)

1120

In [16]:
mini_data = pd.DataFrame(total_info)
mini_data.head()

,Name,Rating,Date,Review
0,Gopchang Story Fort Lee,5 star rating,12/25/2020,It was a delightful experience overall. Ambian...
1,Gopchang Story Fort Lee,5 star rating,12/10/2020,"Great customer service. Most importantly, safe..."
2,Gopchang Story Fort Lee,5 star rating,1/23/2021,The restaurant does not take reservations so w...
3,Gopchang Story Fort Lee,1 star rating,2/6/2021,Used to love this place. But I and my friends ...
4,Gopchang Story Fort Lee,4 star rating,1/22/2021,Finally got to check out the Fort Lee location...


In [17]:
mini_data.to_csv('mini_review_dataset.csv', encoding='utf-8')

### 把同一个餐厅的所有review都并起来

In [43]:
# 这样一家餐厅爬下来的所有review都在一个column里面了
reviews_df = pd.DataFrame(mini_data.groupby('Name')['Review'].sum()).reset_index()
reviews_df

,Name,Review
0,Cap't Loui,I have dined here a few times during the pande...
1,Gamja Tang Tang,PRE COVID REVIEW!!!\n\nAbsolutely love this pl...
2,Gopchang Story Fort Lee,It was a delightful experience overall. Ambian...
3,Lauren’s Chicken Burger,I can not get over how delicious my supreme ch...
4,Marty's,I came here with a friend during COVID times a...
5,OISO BBQ PIT,This BBQ joint is legit. Stick with the staple...
6,Sa Rit Gol,"Atmosphere: 5\nDue to COVID, we weren't able t..."
7,Soba Noodle Azuma,Covid times:\n\nOrdered take out a couple time...
8,Soup Dumpling Plus,I am so happy that I FINALLY have an amazing s...
9,Wok Bar,"I ordered directly through their website, and ..."


In [89]:
reviews = reviews_df.Review

#### check
- '\xa0' -> ' '
- '\n' -> ' '
- 符号：&, -, \$, :, (, ), /, 
- double space 换成 single space
- remove number
- 缩写

In [72]:
import string
from nltk.corpus import stopwords

import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *
from nltk.tokenize import TweetTokenizer, RegexpTokenizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [73]:
# Stopwords
add_stop = ['said', 'say', '...', 'like', 'cnn', 'ad', 'thi', 'ha', 'wa']
stop_words = ENGLISH_STOP_WORDS.union(add_stop)

# Punctuations
punc = list(set(string.punctuation))

# Tokenize
def casual_tokenizer(text):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens

In [74]:
# Contraction
c_dict = {"ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because",
    "could've": "could have","couldn't": "could not","couldn't've": "could not have","didn't": "did not","doesn't": "does not",
    "don't": "do not","hadn't": "had not","hadn't've": "had not have","hasn't": "has not","haven't": "have not","he'd": "he would",
    "he'd've": "he would have","he'll": "he will","he'll've": "he will have","he's": "he is","how'd": "how did","how'd'y": "how do you",
    "how'll": "how will","how's": "how is","i'd": "I would","i'd've": "I would have","i'll": "I will","i'll've": "I will have",
    "i'm": "I am","i've": "I have","isn't": "is not", "it'd": "it had", "it'd've": "it would have","it'll": "it will",
    "it'll've": "it will have","it's": "it is","let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have",
    "mightn't": "might not","mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have",
    "needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not","oughtn't've": "ought not have",
    "shan't": "shall not","sha'n't": "shall not","shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
    "she'll": "she will","she'll've": "she will have","she's": "she is","should've": "should have","shouldn't": "should not",
    "shouldn't've": "should not have","so've": "so have","so's": "so is","that'd": "that would","that'd've": "that would have",
    "that's": "that is","there'd": "there had","there'd've": "there would have","there's": "there is","they'd": "they would",
    "they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are", "they've": "they have",
    "to've": "to have","wasn't": "was not","we'd": "we had", "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
    "we're": "we are","we've": "we have","weren't": "were not","what'll": "what will","what'll've": "what will have",
    "what're": "what are","what's": "what is","what've": "what have","when's": "when is","when've": "when have","where'd": "where did",
    "where's": "where is","where've": "where have", "who'll": "who will","who'll've": "who will have","who's": "who is",
    "who've": "who have","why's": "why is","why've": "why have","will've": "will have","won't": "will not","won't've": "will not have",
    "would've": "would have","wouldn't": "would not","wouldn't've": "would not have","y'all": "you all","y'alls": "you alls",
    "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
    "you'd": "you had","you'd've": "you would have","you'll": "you you will","you'll've": "you you will have","you're": "you are",
    "you've": "you have"
}

# compile the contraction dictionary
c_re = re.compile('(%s)' % '|'.join(c_dict.keys()))
def expandContractions(text, c_re = c_re):
    def replace(match):
        return c_dict[match.group(0)]
    return c_re.sub(replace, text)

In [87]:
# Texing processing
stemmer = PorterStemmer()
def process_text(text):
    # Simple processing
    text = text.replace('\a0', ' ').replace('\n', ' ').replace('.', ' ').replace(',', ' ').replace("'", " ")
    # 1. tokenize
    text = casual_tokenizer(text)
    # 2. lower case
    text = [word.lower() for word in text]
    # 3. remove numbers/digits
    text = [re.sub(r'[0-9]+', '', word) for word in text]
    # 4. replace contractions
    text = [expandContractions(word, c_re) for word in text]
    # 5. stemming
    text = [stemmer.stem(word) for word in text]
    # 6. remove punctuations
    text = [word for word in text if word not in punc]
    # 7. remove stopwords
    text = [word for word in text if word not in stop_words]
    # 8. remove single letter 
    text = [word for word in text if len(word) > 1]
    # 9. clear spaces
    text = [each for each in text if ' ' not in each]
    
    return text

In [90]:
clean_reviews = reviews.apply(process_text)

In [91]:
reviews_df['clean_review'] = clean_reviews
reviews_df

,Name,Review,clean_review
0,Cap't Loui,I have dined here a few times during the pande...,"[dine, time, dure, pandem, taken, food, dine, ..."
1,Gamja Tang Tang,PRE COVID REVIEW!!!\n\nAbsolutely love this pl...,"[pre, covid, review, absolut, love, place, esp..."
2,Gopchang Story Fort Lee,It was a delightful experience overall. Ambian...,"[delight, experi, overal, ambianc, nice, great..."
3,Lauren’s Chicken Burger,I can not get over how delicious my supreme ch...,"[delici, suprem, chicken, burger, lauren, chic..."
4,Marty's,I came here with a friend during COVID times a...,"[came, friend, dure, covid, time, impress, saf..."
5,OISO BBQ PIT,This BBQ joint is legit. Stick with the staple...,"[bbq, joint, legit, stick, stapl, brisket, rib..."
6,Sa Rit Gol,"Atmosphere: 5\nDue to COVID, we weren't able t...","[atmospher, covid, weren, abl, eat, order, pro..."
7,Soba Noodle Azuma,Covid times:\n\nOrdered take out a couple time...,"[covid, time, order, coupl, time, did, disappo..."
8,Soup Dumpling Plus,I am so happy that I FINALLY have an amazing s...,"[happi, final, amaz, spot, chines, food, subje..."
9,Wok Bar,"I ordered directly through their website, and ...","[order, directli, websit, deliveri, came, quic..."


In [92]:
reviews_df.to_csv('clean_review_table.csv', encoding='utf-8')